# Fact Or Opinion

- In this notebook, I prepared a batch to determine whether each thesis-relevant comment expresses a factual statement or a personal opinion.  
- I used the rewritten comment text to build the batch JSONL file.  
- The prompt asked GPT-4o Mini to return `1` if the comment was factual, or `0` if it was an opinion.  
- To stay within the token limit, I split the batch into two parts before submitting it to OpenAI’s Batch API.  
- Once the outputs were ready, I downloaded the JSONL files and extracted the `custom_id` and the fact/opinion label.  
- I saved the results to a CSV to explore how fact-based and opinion-based comments varied across countries, events, and categories.


### Importing Libraries

In [ ]:
import pandas as pd
import json
from tqdm import tqdm
import os
import openai

In [2]:
# Loading the full dataset
df = pd.read_csv("Thesis_Relevant_With_Transcript_Influence_and_Agreement.csv")
print(f"Loaded {len(df)} comments.")

Loaded 71441 comments.


/var/folders/nv/wkg5hpgj1kl1012mfw_qwby80000gn/T/ipykernel_13655/3311419059.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Thesis_Relevant_With_Transcript_Influence_and_Agreement.csv")


### Creating the Json file

In [2]:
output_jsonl = "batch_step_factual_opinion.jsonl"

# Opening the file and writing each comment as a separate task
with open(output_jsonl, "w", encoding="utf-8") as f:
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Building Factual/Opinion batch"):
        comment_id = str(row["Comment_ID"])
        comment = row["Rewritten Comment"]

        # Building the request for GPT-4o Mini
        task = {
            "custom_id": comment_id,  
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o-mini",
                "messages": [
                    {
                        "role": "system",
                        "content": "You are helping a researcher decide whether a comment is stating a fact or expressing an opinion."
                    },
                    {
                        "role": "user",
                        "content": f"""Comment: "{comment}"\n\nDoes this comment state a fact or express an opinion?\n\nReply ONLY with:\n1 = factual\n0 = opinion"""
                    }
                ],
                "temperature": 0,  
                "max_tokens": 5    
            }
        }

        f.write(json.dumps(task) + "\n")

print(f"Saved: {output_jsonl}")


Building Factual/Opinion batch: 100% |█████| 71441/71441 [00:07<00:00, 9540.05it/s]
Saved: batch_step_factual_opinion.jsonl


### Splitting the Batch due to Token Limit

In [6]:
input_file = "batch_step_factual_opinion.jsonl"  
output_file_1 = input_file.replace(".jsonl", "_part1.jsonl")
output_file_2 = input_file.replace(".jsonl", "_part2.jsonl")

# Reading the file
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Splitting
mid = len(lines) // 2
part1 = lines[:mid]
part2 = lines[mid:]

# Writing to two new files
with open(output_file_1, "w", encoding="utf-8") as f:
    f.writelines(part1)

with open(output_file_2, "w", encoding="utf-8") as f:
    f.writelines(part2)

print(f"Saved {len(part1)} → {output_file_1}")
print(f"Saved {len(part2)} → {output_file_2}")


Saved 35720 → batch_step_factual_opinion_part1.jsonl
Saved 35721 → batch_step_factual_opinion_part2.jsonl


### Submitting Batches to Openai

In [8]:
openai.api_key = "***************" 
client = openai.OpenAI(api_key=openai.api_key)

# JSONL batch files
batch_files = [
    "batch_step_factual_opinion_part1.jsonl",
    "batch_step_factual_opinion_part2.jsonl"
]

# Submitting batches
for i, file_path in enumerate(batch_files):
    try:
        print(f"\nSubmitting batch {i+1}/{len(batch_files)}: {file_path}")

        # Uploading file
        with open(file_path, "rb") as f:
            upload = client.files.create(file=f, purpose="batch")
        print(f"Uploaded: File ID = {upload.id}")

        # Creating batch job
        batch = client.batches.create(
            input_file_id=upload.id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
        )
        print(f"Batch submitted: Batch ID = {batch.id}")

    except Exception as e:
        print(f"Failed to submit {file_path}: {e}")


Submitting batch 1/2: batch_step_factual_opinion_part1.jsonl
Uploaded: File ID = file-9FEwxceiWTW895qVphkfgg
Batch submitted: Batch ID = batch_67e5e08d68dc819090955efe8d849f47

Submitting batch 2/2: batch_step_factual_opinion_part2.jsonl
Uploaded: File ID = file-9MMyyk9fEk3Ba2fjtUNZn1
Batch submitted: Batch ID = batch_67e5e0954308190b38a9a00c72cc9960


### Downloading Json Output from Openai

In [16]:
# Set OpenAI API Key
openai.api_key = "**************" 

# List of output file IDs from completed batches
output_file_ids = [
    "file-WUVymZPnxciE6JhhTzrFuF",
    "file-6z3wqTh4SS9jQ59hBeBV1C"
]

# Downloading each output file properly
for file_id in output_file_ids:
    file_response = openai.files.content(file_id)

    # Saving the file locally in binary mode
    output_filename = f"{file_id}.jsonl"
    with open(output_filename, "wb") as f:
        for chunk in file_response.iter_bytes():
            f.write(chunk)
    
    print(f"File downloaded: {output_filename}")

File downloaded: file-WUVymZPnxciE6JhhTzrFuF.jsonl
File downloaded: file-6z3wqTh4SS9jQ59hBeBV1C.jsonl


In [18]:
# List of JSONL files for Factual/Opinion
jsonl_files = [
    "file-WUVymZPnxciE6JhhTzrFuF.jsonl",
    "file-6z3wqTh4SS9jQ59hBeBV1C.jsonl"
]

# Storing extracted data
extracted_data = []

# Looping through files
for jsonl_file in jsonl_files:
    if not os.path.exists(jsonl_file):
        print(f"{jsonl_file} not found")
        continue

    print(f"Processing {jsonl_file}")

    with open(jsonl_file, "r", encoding="utf-8") as f:
        for line in f:
            json_obj = json.loads(line)

            custom_id = json_obj.get("custom_id", "")
            response = json_obj.get("response", {}).get("body", {}).get("choices", [{}])[0].get("message", {}).get("content", "").strip()

            extracted_data.append({
                "custom_id": custom_id,
                "Factual_or_Opinion": response
            })

# Creating and save CSV
df = pd.DataFrame(extracted_data)
output_csv = "factual_or_opinion_responses.csv"
df.to_csv(output_csv, index=False)

# Summary counts
factual = (df["Factual_or_Opinion"] == "1").sum()
opinion = (df["Factual_or_Opinion"] == "0").sum()

print(f"\nExtracted responses saved to: {output_csv}")
print(f"Factual : {factual}")
print(f"Opinion : {opinion}")


Processing file-WUVymZPnxciE6JhhTzrFuF.jsonl
Processing file-6z3wqTh4SS9jQ59hBeBV1C.jsonl

Extracted responses saved to: factual_or_opinion_responses.csv
Factual : 3763
Opinion : 67678
